<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

Импорт необходимых библиотек

In [1]:
!pip install imblearn -q
!pip install ydata_profiling -q
!pip install pymystem3

You should consider upgrading via the '/Users/polina.piskovatskova/Documents/YP/projects/practicum_env/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/polina.piskovatskova/Documents/YP/projects/practicum_env/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/polina.piskovatskova/Documents/YP/projects/practicum_env/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import os
import warnings
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import transformers
from tqdm import notebook
import re
import nltk
from nltk.corpus import stopwords as nltk_stopwords




from sklearn.model_selection import (cross_val_score,
                                     RandomizedSearchCV,
                                     train_test_split)
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from pymystem3 import Mystem
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from catboost import CatBoostClassifier
from ydata_profiling import ProfileReport

RANDOM_STATE=42

/Users/polina.piskovatskova/Documents/YP/projects/practicum_env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
file_path = '/datasets/toxic_comments.csv'
if os.path.exists(file_path):
    comments = pd.read_csv(file_path)
else:
    comments = pd.read_csv('/Users/polina.piskovatskova/Documents/YP/datasets/machine_learning_for_texts/toxic_comments.csv')


Изучим данные, проверим наличие пропусков, дубликатов, баланс классов.

In [4]:
profile = ProfileReport(comments, title="Profiling Report")
display(profile)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

В датасете нет пропусков, но есть не несущий пользы столбец 'Unnamed', дублирующий индексы. Удалим его.

In [5]:
comments = comments[['text', 'toxic']]
comments.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Предположим, что большая часть комментариев на английском языке. Оставим только комментарии на английском языке, чтобы использовать предобученную модель BERT.

In [6]:
# Фильтруем строки, содержащие символы латинского алфавита и другие символы
comments_latin = comments[comments['text'].str.contains(r'^[a-zA-Z0-9\s\.,!?;:\'"-]+$', regex=True)]
comments_latin.info()


<class 'pandas.core.frame.DataFrame'>
Index: 86288 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    86288 non-null  object
 1   toxic   86288 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.0+ MB


Подробнее изучим баланс классов.

In [7]:
class_counts = comments_latin["toxic"].value_counts()

# Визуализация дисбаланса классов
plt.figure(figsize=(6, 4))
class_counts.plot(kind="bar", color=["blue", "red"])
plt.xticks(ticks=[0, 1], labels=["Non-Toxic (0)", "Toxic (1)"], rotation=0)
plt.ylabel("Count")
plt.title("Class Distribution")
plt.show()

# Выведем соотношение классов
class_counts_normalized = class_counts / class_counts.sum()
class_counts, class_counts_normalized

/var/folders/mb/tdzwqv9137jby082pkr2y4c00000gn/T/ipykernel_53971/1997986395.py:9: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


(toxic
 0    73904
 1    12384
 Name: count, dtype: int64,
 toxic
 0    0.856481
 1    0.143519
 Name: count, dtype: float64)

Токсичных комментариев в 5 раз меньше, чем нетоксичных. Поэтому при обучении моделей введем параметр взвешивания классов, чтобы модель более точно предсказывала минорный класс.

Вычислим максимальную длину текста комментария для дальнейшей обработки. Токенизатор BERT принимает максимум 512 токенов.

In [8]:
# Функция для вычисления длины текста в токенах
def count_tokens(text):
    return len(text)

# Вычисление длины текста в токенах комментариев
text_length = comments_latin['text'].apply(count_tokens)
max_text_length = text_length.max()
print(f"Максимальная длина текста в комментарии: {max_text_length}")

Максимальная длина текста в комментарии: 5000


Попробуем использовать предобученную модель BERT для англоязычных токсичных комментариев, toxic-bert, с ограничением длины текста. В целях проверки обучения модели выберем небольшое количество твитов.

Я также пробовала использовать предобученную модель BERT для англоязычных текстов, bert-base-uncased, но не получила хорошей метрики F1 (максимум 0.71 на тестовой выборке). Наверное, для улучшения показателей нужно было увеличить размер выборки для обучения, но тогда обучение заняло бы слишком много времени. Также на низкие показатели может влиять максимальный размер текста для токенизатора BERT - 512 символов, важная часть комментария может быть обрезана.

In [9]:
# Выбор 5000 случайных семплов
sample_size = 5000
comments_small = comments_latin.sample(n=sample_size, random_state=RANDOM_STATE).reset_index(drop=True)

In [10]:
# Загрузка токенизатора и модели по имени
tokenizer = transformers.BertTokenizer.from_pretrained('unitary/toxic-bert')
model = transformers.BertModel.from_pretrained('unitary/toxic-bert')

In [11]:
# Токенизация текста
tokenized = comments_small['text'].apply(
    lambda x: tokenizer.encode(x,
                               add_special_tokens=True,
                               max_length=512,
                               truncation=True
                               )
)

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

In [12]:
# Получение эмбеддингов BERT
batch_size = 50
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/100 [00:00<?, ?it/s]

In [13]:
features = np.concatenate(embeddings)
target = comments_small['toxic'].values

# Разделение данных на обучающую и тестовую выборки (50:50)
X_train, X_test, y_train, y_test = train_test_split(features,
                                                    target,
                                                    test_size=0.5,
                                                    random_state=RANDOM_STATE,
                                                    stratify=target)


## Обучение

Обучим модели с помощью рандомизированного поиска.

In [15]:
# Пайплайн: модель
pipe_final = Pipeline([
    ('models', DecisionTreeClassifier(random_state=RANDOM_STATE, class_weight='balanced'))
])

# Зададим гиперпараметры моделей
parameters_grid = [
    # словарь для модели DecisionTreeClassifier
    {
    'models': [DecisionTreeClassifier(random_state=RANDOM_STATE)],
    'models__class_weight': [1,7],
    'models__max_features': range(2,6),
    'models__min_samples_leaf': range(2,4),
    'models__min_samples_split':range(2,16)
    },
    # словарь для модели RandomForest
    {
    'models':[RandomForestClassifier(n_estimators=100, random_state=RANDOM_STATE)],
    'models__class_weight': [1,7],
    'models__n_estimators': [50, 100, 200],
    'models__max_depth': [10, 20, 30],
    },
    # словарь для модели LogisticRegression
    {
    'models': [LogisticRegression(random_state=RANDOM_STATE, class_weight='balanced', max_iter=1000)],
    'models__C': [0.1,0.4,0.5,0.6,0.7,0.8,1,2,5,10],
    'models__penalty': ['l1', 'l2', 'elasticnet', None],
    'models__solver': ['lbfgs', 'liblinear', 'sag', 'saga']
    },
    # словарь для модели LightGBM
    {
    'models': [lgb.LGBMClassifier(n_estimators=100, random_state=RANDOM_STATE)],
    'models__class_weight': [1,7],
    'models__n_estimators': [50, 100, 200],
    'models__learning_rate': [0.01, 0.1, 0.2]
    },
    # словарь для модели CatBoost
    {
    'models': [CatBoostClassifier(random_state=RANDOM_STATE, verbose=0, loss_function='F1', class_weights='balanced')],
    'models__iterations': [100, 200, 300],
    'models__learning_rate': [0.01, 0.1, 0.2],
    'models__depth': [4, 6, 8]
    }
]

In [16]:
warnings.filterwarnings("ignore", category=RuntimeWarning)

# Используем рандомизированный поиск для подбора лучшей модели по метрике F1
randomized_search = RandomizedSearchCV(
    pipe_final,
    parameters_grid,
    cv=5,
    scoring='f1',
    random_state=RANDOM_STATE,
    n_jobs=-1
)
randomized_search.fit(X_train, y_train)


print('Лучшие параметры модели:\n\n', randomized_search.best_params_)
print('Лучшая метрика модели на тренировочных данных:\n', randomized_search.best_score_)

/Users/polina.piskovatskova/Documents/YP/projects/practicum_env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/polina.piskovatskova/Documents/YP/projects/practicum_env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/polina.piskovatskova/Documents/YP/projects/practicum_env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/polina.piskovatskova/Documents/YP/projects/practicum_env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/polina.piskovatskova/Documents/YP/projects/practicum_env/lib/python3.9/site-packages/sklearn/

Лучшие параметры модели:

 {'models__solver': 'saga', 'models__penalty': 'l2', 'models__C': 5, 'models': LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)}
Лучшая метрика модели на тренировочных данных:
 0.9480705348169124


/Users/polina.piskovatskova/Documents/YP/projects/practicum_env/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [17]:
y_pred = randomized_search.best_estimator_.predict(X_test)
f1_test = f1_score(y_test, y_pred)
print(f'F1 на тестовой выборке: {f1_test}')

F1 на тестовой выборке: 0.9446808510638298


Метрика F1 > 0.75, что соответствует условию задачи и является высоким результатом, что подтверждает важность учета контекста слов в задачах классификации текстов.
Метрика не должна быть слишком высокой, так как это свидетельствовало бы об утечке данных.  

Лучшая модель Логистическая регрессия и ее параметры:  
- Solver: saga,
- Регуляризация l2, сила регуляризации С: 5,
- Взвешивание классов class_weight='balanced', max_iter=1000, random_state=42.


## Альтернативный вариант классификации с использованием TF-IDF

Проверим, как обучатся модели для бинарной классификации, если не использовать векторную модель, учитывающую смысловую связь слов между собой.

In [18]:
# Скопируем уменьшенный датасет с англоязычными комментариями
df = comments_small.copy()

# Инициализация MyStem для лемматизации текста
m = Mystem()

# Лемматизируем текст комментариев
def lemmatize(text):
    lemm_list = m.lemmatize(text)
    lemm_text = "".join(lemm_list)
        
    return lemm_text

# Подготовим чистый текст
def clear_text(text):
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = text.lower()
    text = " ".join(text.split())
    return text

In [19]:
# Очистка текста
df['cleared_text'] = df['text'].apply(clear_text)

# Лемматизация текста
df['lemmatized_text'] = df['cleared_text'].apply(lemmatize)

display(df[['text', 'cleared_text', 'lemmatized_text']].head())


,text,cleared_text,lemmatized_text
0,Why do you get so angry when I edit sailor moo...,why do you get so angry when i edit sailor moo...,why do you get so angry when i edit sailor moo...
1,Australian Idol 2007 \n\nWasn't she on Idol as...,australian idol 2007 wasnt she on idol as a ho...,australian idol 2007 wasnt she on idol as a ho...
2,Reporting you\nI now reporting you. Despite be...,reporting you i now reporting you despite bein...,reporting you i now reporting you despite bein...
3,"""\nSometimes it is while other times it isn't,...",sometimes it is while other times it isnt also...,sometimes it is while other times it isnt also...
4,IM GLAD THAT THE MOTHERFUCKERS WIFE DIED! FILL...,im glad that the motherfuckers wife died fills...,im glad that the motherfuckers wife died fills...


In [20]:
# Загрузим список стоп-слов
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/polina.piskovatskova/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
stopwords = list(nltk_stopwords.words('english'))

In [22]:
# Подготовим входные и целевой признаки
features = df['lemmatized_text'].values.astype('U')
target = df['toxic'].values

In [23]:
# Разделим признаки на обучающую и тестовую в соотношении 50:50
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(
        features, target, test_size=0.5, random_state=RANDOM_STATE)

In [24]:
# Инициализация TF-IDF
tfidf = TfidfVectorizer(stop_words=stopwords)

# Преобразование текста в TF-IDF признаки
X_train_tfidf = tfidf.fit_transform(X_train_tfidf)
X_test_tfidf = tfidf.transform(X_test_tfidf)

In [25]:
warnings.filterwarnings("ignore", category=RuntimeWarning)
# Используем рандомизированный поиск для подбора лучшей модели по метрике F1
randomized_search_tfidf = RandomizedSearchCV(
    pipe_final,
    parameters_grid,
    cv=5,
    scoring='f1',
    random_state=RANDOM_STATE,
    n_jobs=-1
)
randomized_search_tfidf.fit(X_train_tfidf, y_train_tfidf)


print('Лучшие параметры модели:\n\n', randomized_search_tfidf.best_params_)
print('Лучшая метрика модели на тренировочных данных:\n', randomized_search_tfidf.best_score_)

/Users/polina.piskovatskova/Documents/YP/projects/practicum_env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/polina.piskovatskova/Documents/YP/projects/practicum_env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/polina.piskovatskova/Documents/YP/projects/practicum_env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/polina.piskovatskova/Documents/YP/projects/practicum_env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/polina.piskovatskova/Documents/YP/projects/practicum_env/lib/python3.9/site-packages/sklearn/

Лучшие параметры модели:

 {'models__solver': 'sag', 'models__penalty': None, 'models__C': 0.8, 'models': LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)}
Лучшая метрика модели на тренировочных данных:
 0.6337854094846277


In [26]:
y_pred_tfidf= randomized_search_tfidf.best_estimator_.predict(X_test_tfidf)
f1_test = f1_score(y_test_tfidf, y_pred_tfidf)
print(f'F1 на тестовой выборке TF-IDF: {f1_test}')

F1 на тестовой выборке TF-IDF: 0.6927710843373494


Результат получился хуже, чем при векторизации текста.

## Выводы

В результате работы над задачей выполнены следующие шаги:  

1. **Подготовка данных:**  
 - Данные загружены и проанализированы. Удалены ненужные столбцы, а также отфильтрованы комментарии на английском языке для дальнейшей работы.  
- Проведен анализ баланса классов, который показал, что токсичные комментарии составляют меньшинство (около 14% от общего числа). Это было учтено при обучении моделей для корректной работы с дисбалансом классов.

2. **Обучение моделей:**  
Для классификации токсичных комментариев была использована предобученная модель BERT (Toxic-BERT), которая учитывает контекст слов и демонстрирует высокую точность в задачах обработки естественного языка.  
С помощью рандомизированного поиска по гиперпараметрам были подобраны оптимальные настройки для различных моделей.  
Лучший результат показала логистическая регрессия с параметрами: solver='saga', penalty='l2', C=5, class_weight='balanced'.
Метрика F1 на тестовой выборке для модели с использованием BERT составила 0.944, что превышает требуемый порог в 0.75.
Возможно, результат можно было бы улучшить при увеличении размера обучающей выборки, но тогда это заняло бы слишком много времени и потребовало бы больших вычислительных ресурсов.

3. **Альтернативный подход с использованием TF-IDF:**   
В качестве альтернативы был рассмотрен подход с использованием TF-IDF для векторизации текста. Этот метод не учитывает контекст слов, но требует меньше вычислительных ресурсов.   
Лучшая модель с использованием TF-IDF также оказалась логистической регрессией, но с параметрами: solver='sag', penalty=None, C=0.8.  
Метрика F1 на тестовой выборке для этого подхода составила 0.693, что ниже, чем у модели с использованием BERT, но всё же является приемлемым результатом.   

Модель на основе BERT показала значительно более высокие результаты, что подтверждает важность учета контекста слов в задачах классификации текстов.
Подход с использованием TF-IDF может быть полезен, если вычислительные ресурсы ограничены, но его результаты уступают более сложным моделям.